## Выбираем лучшие настройки для тикеров

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from bot.vis import TaskProgress
from bot import TestAlgorithm
from app.config import RunConfig
from concurrent.futures import ThreadPoolExecutor, as_completed

test_configs = [
    (RunConfig(
        ticker=t_config.ticker,
        step_max_cnt=max_shares,
        step_base_cnt=base_shares,
        step_lots=1,

        majority_trade=t_config.majority_trade,
        pretest_period=pretest_period,
        pretest_type=RunConfig.PRETEST_RSI,  # todo

        threshold_buy_steps=0,
        threshold_sell_steps=0,
        stop_up_p=stop_up_p,
        stop_down_p=0,

        step_size=t_config.step_size + step_size_shift,
        step_set_orders_cnt=step_cnt,
    ))
    for t_config in [
        # RunConfig.from_repr_string('DELI+ 3/pre6:0/2 x l2 x 1.6¤ |u0.01 d0.0|'),
        # RunConfig.from_repr_string('ETLN- 3/pre4:0/2 x l10 x 1.0¤'),
        # RunConfig.from_repr_string('EUTR+ 3/pre3:-3/3 x l2 x 0.8¤'),
        # RunConfig.from_repr_string('RNFT+ 3/pre5:3/2 x l2 x 1.4¤'),
        # RunConfig.from_repr_string('SPBE- 3/pre6:0/2 x l10 x 1.8¤'),


        # RunConfig.from_repr_string('GEMC- 3/pre6:0/2 x l10 x 2.4¤'),
        # RunConfig.from_repr_string('HNFG- 3/rsi13:0/2 x l10 x 3.2¤'),
        RunConfig.from_repr_string('UWGN- 3/pre6:0/2 x l10 x 1.2¤'),
        RunConfig.from_repr_string('GLTR- 3/pre6:0/2 x l10 x 5.4¤'),
        RunConfig.from_repr_string('POLY- 3/pre6:0/2 x l10 x 3.4¤'),
        RunConfig.from_repr_string('ASTR- 3/pre6:0/2 x l10 x 1.4¤'),
        RunConfig.from_repr_string('QIWI- 3/pre6:0/2 x l10 x 2.6¤'),
    ]
    for max_shares in [3]
    for base_shares in [0]
    for stop_up_p in [0]
    for step_size_shift in [0, .2, -.2, -.4, .4]
    for step_cnt in [2]  # вот тут некоторым удобнее в 2. можно потестить в вариаторе 2 и 3
    for pretest_period in [13]  # range(3, 7) # todo
]


def run_test(config: RunConfig):
    test_alg = TestAlgorithm(do_printing=False, config=config)
    return test_alg.test(
        last_test_date=None,
        test_days_num=30,  # todo
        shares_count=0,

        # auto_conf_days_freq=1,
        # auto_conf_prev_days=config.pretest_period,
    )

unique_configs = set(test_configs)
# unique_configs = test_configs

results = []
progress = TaskProgress(len(unique_configs))


# todo на первом проходе надо 1 ставить, тогда запросы к API будут только 1 раз
with ThreadPoolExecutor(max_workers=4) as executor:
    future_to_params = {executor.submit(run_test, config): config for config in unique_configs}
    
    for future in as_completed(future_to_params):
        params = future_to_params[future]
        res = future.result()
        if res:
            results.append(res)
        progress.update_progress()

# Вывод результатов или их дальнейшая обработка
sorted_results = sorted(results, key=lambda x: (x['config'].ticker, -float(x['profit_p'])), reverse=False)

print()
for item in sorted_results:
    print(item)

In [ ]:
a = 2

#82 Динамическая подстройка параметров запуска
---

исходный вариант алгоритма
{'profit': 549.7, 'profit_p': 55.41, 'config': RNFT 4/pre13:0/3 x l2 x 1.2¤, |s0 b0| |u0.0 d0.0| maj+z+ , 'op_cnt': 2318}

на pre
{'profit': 253.8, 'profit_p': 51.17, 'config': RNFT 4/pre13:0/3 x l1 x 1.2¤, |s0 b0| |u0.0 d0.0| maj+z+ , 'op_cnt': 2318}

last_test_date='2024-05-01',
test_days_num=365,  # 365,
auto_conf_days_freq=7,
auto_conf_prev_days=7,
{'profit': 42.5, 'profit_p': 8.57, 'config': RNFT 4/0/3 x l1 x 1.2¤, |s0 b0| |u0.0 d0.0| maj+z+ , 'op_cnt': 2283}



